In [1]:
import sys
sys.path = ["/home/jmetzen/Repositories/scikit-learn", 
            "/home/jmetzen/Repositories/team/src/bayesian_optimization"] + sys.path
import cPickle

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
#from mpl_toolkits.axes_grid1.inset_locator import inset_axes
#import seaborn as sns
%matplotlib inline

plt.rcParams.update({'axes.labelsize': 11,
                     'axes.titlesize': 12,
                     'text.fontsize': 11,
                     'xtick.labelsize': 8,
                     'ytick.labelsize': 8,
                     'legend.fontsize': 7,
                     'legend.fancybox': True,
                     'font.family': 'serif'})

c = ["r", "g", "b", 'k', 'm', 'c']
ms = ["o", "d", "v", "^", "s", "p"]

from joblib import Parallel, delayed

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic

from bayesian_optimization import (BayesianOptimizer, GaussianProcessModel, 
                                   MinimalRegretSearch, EntropySearch, UpperConfidenceBound,
                                   ExpectedImprovement, ProbabilityOfImprovement,
                                   GPUpperConfidenceBound)
from bayesian_optimization.utils.optimization import global_optimization

/home/jmetzen/.anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/home/jmetzen/.anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: text.fontsize is deprecated and replaced with font.size; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
def generate_function(kernel, boundaries, n_train_points, random_state):
    gp_prior = GaussianProcessRegressor(kernel=kernel)

    X_train = np.random.RandomState(random_state).uniform(
        boundaries[:, 0], boundaries[:, 1], (n_train_points, boundaries.shape[0]))
    y_train = gp_prior.sample_y(X_train, 1, random_state=random_state)

    gp_target = GaussianProcessRegressor(kernel=kernel)
    gp_target.fit(X_train, y_train)
    
    def target_fct(x):
        return gp_target.predict(x[np.newaxis, :])[0, 0]
    
    return target_fct

In [3]:
def create_optimizer(kernel, acquisition_function):
    model = GaussianProcessModel(kernel=kernel, alpha=1e-3)
    if acquisition_function == "mrs":
        acquisition_function = \
            MinimalRegretSearch(model=model, n_gp_samples=1000, n_candidates=25,
                                n_trial_points=250, n_samples_y=51, point=False)
    if acquisition_function == "mrs_point":
        acquisition_function = \
            MinimalRegretSearch(model=model, n_gp_samples=1000, n_candidates=25,
                                n_trial_points=250, n_samples_y=51, point=True)
    elif acquisition_function == "es":
        acquisition_function = \
            EntropySearch(model=model, n_gp_samples=1000, n_candidates=25,
                          n_trial_points=250, n_samples_y=51)
    elif acquisition_function == "ucb":
        acquisition_function = \
            UpperConfidenceBound(model=model, kappa=5.0)
    elif acquisition_function == "gp_ucb":
        acquisition_function = \
            GPUpperConfidenceBound(model=model, const=0.5)
    elif acquisition_function == "ei":
        acquisition_function = \
            ExpectedImprovement(model=model)
    elif acquisition_function == "pi":
        acquisition_function = \
            ProbabilityOfImprovement(model=model)
    bayes_opt = BayesianOptimizer(model=model, optimizer="direct+lbfgs",
                                  acquisition_function=acquisition_function)
    return bayes_opt

In [4]:
kernel = RationalQuadratic(length_scale=0.1, length_scale_bounds="fixed", alpha=1.0, alpha_bounds="fixed")
kernel_bo = RBF(length_scale=0.1, length_scale_bounds="fixed")

n_train_points = 250
boundaries = np.array([[0.0, 1.0], [0.0, 1.0]])

n_runs = 250
n_trials = 100
settings = ["mrs", "es", "mrs_point", "ei", "gp_ucb", "pi"]

In [5]:
def perform_run(setting, run):
    np.random.seed(run)
    
    y_regret = np.empty((n_trials / 5))
    X_dist = np.empty((n_trials / 5))
    X_query = np.empty((n_trials, boundaries.shape[0]))
    
    target_fct = generate_function(kernel, boundaries, n_train_points, random_state=run)
    X_opt = global_optimization(target_fct,  boundaries=boundaries, 
                                optimizer="direct", maxf=1000)
    y_opt = target_fct(X_opt) 

    bayes_opt = create_optimizer(kernel_bo,acquisition_function=setting)

    for i in range(n_trials):
        query = bayes_opt.select_query_point(boundaries)
        result = target_fct(query) + 1e-3 * np.random.random()
        bayes_opt.update(query, result)
        
        X_query[i] = query
        if i % 5 == 4:
            X_sel = global_optimization(lambda x: bayes_opt.model.gp.predict(x[np.newaxis, :]), 
                                        boundaries=boundaries, 
                                        optimizer="direct", maxf=1000)
            y_sel = target_fct(X_sel)

            y_regret[i / 5] = y_opt - y_sel
            X_dist[i / 5] = np.sqrt(((X_opt - X_sel)**2).sum())
            
    f = open("/home/jmetzen/Temp/log/%s_%s" % (setting, run), 'w')
    cPickle.dump((setting, run, y_regret, X_dist, X_query), f, protocol=-1)
    f.close()
    return setting, run, y_regret, X_dist, X_query

In [6]:
res = Parallel(n_jobs=4, verbose=10)(
        delayed(perform_run)(setting, run)
            for run in range(n_runs)
                for setting in settings)

[Parallel(n_jobs=4)]: Done   1 out of 1500 | elapsed: 100.5min remaining: 150655.9min
[Parallel(n_jobs=4)]: Done 143 out of 1500 | elapsed: 1178.6min remaining: 11184.4min
[Parallel(n_jobs=4)]: Done 294 out of 1500 | elapsed: 2422.0min remaining: 9935.1min
[Parallel(n_jobs=4)]: Done 445 out of 1500 | elapsed: 3747.1min remaining: 8883.7min
/home/jmetzen/Repositories/team/src/bayesian_optimization/bayesian_optimization/acquisition_functions.py:356: RuntimeWarning: covariance is not positive-semidefinite.
  f_mean, f_cov, self.n_gp_samples).T
/home/jmetzen/Repositories/team/src/bayesian_optimization/bayesian_optimization/acquisition_functions.py:415: RuntimeWarning: covariance is not positive-semidefinite.
  f_mean, f_cov + f_cov_delta, self.n_gp_samples).T
/home/jmetzen/Repositories/team/src/bayesian_optimization/bayesian_optimization/acquisition_functions.py:415: RuntimeWarning: covariance is not positive-semidefinite.
  f_mean, f_cov + f_cov_delta, self.n_gp_samples).T
/home/jmetzen/R

KeyboardInterrupt: 

In [ ]:
y_regret = np.empty((len(settings), n_runs, n_trials / 5))
X_dist = np.empty((len(settings), n_runs, n_trials / 5))
X_query = np.empty((len(settings), n_runs, n_trials, 2))
for setting, run, y_regret_, X_dist_, X_query_ in res:
    i = settings.index(setting)
    y_regret[i, run] = y_regret_
    X_dist[i, run] = X_dist_
    X_query[i, run] = X_query_

In [ ]:
f = open("/home/jmetzen/Repositories/BesMan/SVN/Documentation/Publications/PeerReviewed/drafts/2016_ICML/results/merged_res_200.pickle", "w")
cPickle.dump((y_regret, X_dist, X_query), f)
f.close()

In [ ]:
import cPickle
f = open("/home/jmetzen/Repositories/BesMan/SVN/Documentation/Publications/PeerReviewed/drafts/2016_ICML/results/merged_res_200.pickle", "r")
y_regret, X_dist, X_query = cPickle.load(f)
f.close()

## Plotting

In [ ]:
setting_names = {
    "mrs": "MRS",
    "es": "ES",
    "mrs_point": r"MRS$^{point}$",
    "ei": "EI",
    "gp_ucb": "GP-UCB",
    "pi": "PI"
}

In [ ]:
fig_width = 487.8225 / 72.27  # Get fig_width_pt from LaTeX using \showthe\columnwidth
fig_height = fig_width * 0.5  # height in inches
plt.figure(0, dpi=400, figsize=(fig_width, fig_height))

ax = plt.subplot(1,2,1)
for i, setting in enumerate(settings):
    ax.plot(np.arange(5, n_trials+1, 5), np.median(y_regret[i], 0), 
            c=c[i], marker=ms[i], label=setting_names[setting])
ax.set_ylim(1e-5, 1e1)
#ax.tick_params(axis='y', which='minor')
#ax.yaxis.set_minor_locator(MultipleLocator(5))
ax.set_yscale("log")
ax.set_xlabel("N")
ax.set_ylabel("Regret")
ax.grid(b=True, which='major', color='gray', linestyle='--')
ax.grid(b=True, which='minor', color='gray', linestyle=':')
ax.legend(loc="best", ncol=3, )
ax.set_title("Median Regret")

ax = plt.subplot(1,2,2)
for i, setting in enumerate(settings):
    ax.plot(np.arange(5, n_trials+1, 5), np.mean(y_regret[i], 0), 
            c=c[i], marker=ms[i], label=setting_names[setting])
ax.set_ylim(1e-5, 1e1)
ax.set_yscale("log")
ax.set_xlabel("N")
ax.set_yticklabels([])
ax.grid(b=True, which='major', color='gray', linestyle='--')
ax.grid(b=True, which='minor', color='gray', linestyle=':')
#ax.legend(loc="upper center", ncol=3)
ax.set_title("Mean Regret")

#axins = inset_axes(ax, width="40%", height="40%", loc=1)  # zoom = 6
#axins.scatter(y_regret[0, :, -1], y_regret[1, :, -1], c='r')
#axins.set_xscale("log")
#axins.set_yscale("log")
#axins.set_xlim(1e-10, 1e0)
#axins.set_ylim(1e-10, 1e0)
#axins.set_xticks(np.logspace(-10, 0, 6))
#axins.set_yticks(np.logspace(-10, 0, 6))
#axins.set_axis_bgcolor("white")
#axins.plot([1e-10, 1e0], [1e-10, 1e0])
#axins.set_xlabel("MRS Regret")
#axins.set_ylabel("ES Regret")

plt.tight_layout()
plt.savefig("../pics/empirical_comparison.png", dpi=400)

In [ ]:
fig_width = 487.8225 / 72.27  # Get fig_width_pt from LaTeX using \showthe\columnwidth
fig_height = fig_width * 0.4  # height in inches
plt.figure(0, dpi=400, figsize=(fig_width, fig_height))

for i, setting in enumerate(settings):
    ax = plt.subplot(2, 3, i+1)
    ax.hist(np.maximum(y_regret[i, : , -1], 1e-8), #normed=True,
            bins=np.logspace(-8, 1, 19), color=c[i])
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_title(setting_names[setting])
    ax.set_xticks([1e-8, 1e-6, 1e-4, 1e-2, 1e0])
    if i % 3 != 0:
        ax.set_yticklabels([])
    else:
        ax.set_ylabel("Count")
    if i < 3:
        ax.set_xticklabels([])
    else:
        ax.set_xlabel("Regret")
plt.tight_layout()
plt.savefig("../pics/hist.png", dpi=400)

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(len(settings)):
    for j in range(len(settings)):
        plt.subplot(len(settings), len(settings), j*len(settings) + i + 1)
        plt.scatter(y_regret[i, :, -1], y_regret[j, :, -1], c='r')
        plt.xscale("log")
        plt.yscale("log")
        plt.xlim(1e-10, 1e1)
        plt.ylim(1e-10, 1e1)
        plt.plot([1e-10, 1e1], [1e-10, 1e1], c='k')
        if j == len(settings) - 1:
            plt.xlabel(setting_names[settings[i]])
        plt.gca().set_xticklabels([])
        if i == 0:
            plt.ylabel(setting_names[settings[j]])
        plt.gca().set_yticklabels([])
        plt.title(str((y_regret[i, : , -1] < y_regret[j, : , -1]).mean()))
#(y_regret[0, :, -1] < y_regret[5, :, -1]).sum()
plt.tight_layout()
plt.savefig("../pics/scatter.pdf")

In [ ]:
print "Runs with high immediate regret", zip(settings, (y_regret[:, :, -1] > 1e-2).sum(1))

In [ ]:
# Plt samples on specific runs
run = 0 # y_regret[1, :, -1].argmax()
print run

#kernel = RBF(length_scale=0.1, length_scale_bounds="fixed")
kernel = RationalQuadratic(length_scale=0.1, length_scale_bounds="fixed", alpha=1.0, alpha_bounds="fixed")

n_train_points = 250
boundaries = np.array([[0.0, 1.0], [0.0, 1.0]])
target_fct = generate_function(kernel, boundaries, n_train_points, random_state=run)
X_opt = global_optimization(target_fct,  boundaries=boundaries, 
                            optimizer="direct", maxf=5000)

x_ = np.linspace(0, 1, 50)
y_ = np.linspace(0, 1, 49)
X_, Y_ = np.meshgrid(x_, y_)
Z_ = [[target_fct(np.array([x_[i], y_[j]]))
       for i in range(x_.shape[0])] 
      for j in range(y_.shape[0])]

plt.figure(figsize=(10, 5))
for i in [0]: # range(len(settings)):
    plt.subplot(2, 3, i+1)
    plt.pcolor(X_, Y_, np.array(Z_), cmap = cm.Greys)
    plt.colorbar()
    #plt.scatter(X_query[i, run, :, 0], X_query[i, run, :, 1], c='r')
    #plt.scatter(X_opt[[0]], X_opt[1], c='b')
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    #plt.title(setting_names[settings[i]])
plt.tight_layout()
#plt.savefig("../pics/run_visualization.pdf")

In [ ]:
np.random.seed(0)

setting_index = 1
run = y_regret[setting_index, :, -1].argmax()

target_fct = generate_function(kernel, boundaries, n_train_points, random_state=run)
X_opt = global_optimization(target_fct,  boundaries=boundaries, 
                            optimizer="direct", maxf=5000)

model = GaussianProcessModel(kernel=kernel, alpha=1e-3)
y = [target_fct(X_query[1, run, i]) for i in range(100)]
model.fit(X_query[1, run], y)

n_gp_samples = 1000

mrs = MinimalRegretSearch(model=model, n_gp_samples=n_gp_samples, n_candidates=25,
                          n_trial_points=250, n_samples_y=51, point=False)
mrs.set_boundaries(boundaries)
es = EntropySearch(model=model, n_gp_samples=n_gp_samples, n_candidates=25, 
                   n_trial_points=250, n_samples_y=51)
es.set_boundaries(boundaries, X_candidate=mrs.X_candidate)

x_ = np.linspace(0, 1, 100)
y_ = np.linspace(0, 1, 100)
X_, Y_ = np.meshgrid(x_, y_)

# True function
Z_ = [[target_fct(np.array([x_[i], y_[j]]))
       for i in range(x_.shape[0])] 
      for j in range(y_.shape[0])]

fig_width = 487.8225 / 72.27 # Get fig_width_pt from LaTeX using \showthe\columnwidth
fig_height = fig_width * 0.33  # height in inches
plt.figure(0, dpi=400, figsize=(fig_width, fig_height))

ax = plt.subplot(1, 4, 1)
ax.set_axis_bgcolor('white')
plt.pcolor(X_, Y_, np.array(Z_), cmap = cm.Greys)
#plt.colorbar()
samples = plt.scatter(X_query[setting_index, run, :, 0], X_query[setting_index, run, :, 1],
                      alpha=1.0, c='g', marker='o', label="Samples")
optimum = plt.scatter(X_opt[[0]], X_opt[1],
                      alpha=1.0, c='b', marker='d', s=40, label="Optimum")
plt.xlim(0, 1)
plt.ylim(-0.2, 1)
plt.xticks([])
plt.yticks([])
plt.title("Function/Samples", fontsize=10)

ax = plt.subplot(1, 4, 2)
ax.set_axis_bgcolor('white')
plt.pcolor(X_, Y_, np.array(Z_), cmap = cm.Greys)
repr_points = plt.scatter(mrs.X_candidate[:, 0], mrs.X_candidate[:, 1], 
                          alpha=1.0, c='r', label="Representer points")
plt.legend(handles=[samples, repr_points, optimum], loc="lower right", ncol=3)
plt.xlim(0, 1)
plt.ylim(-0.2, 1)
plt.xticks([])
plt.yticks([])
plt.title("Representer points", fontsize=10)

for index, acq in enumerate([es, mrs]):
    ax = plt.subplot(1, 4, index+3)
    ax.set_axis_bgcolor('white')
    Z_ = [[acq(np.array([x_[i], y_[j]]))[0]
       for i in range(x_.shape[0])] 
      for j in range(y_.shape[0])]

    plt.pcolor(X_, Y_, np.array(Z_), cmap = cm.Greys)
    #plt.colorbar()
    plt.xlim(0, 1)
    plt.ylim(-0.2, 1)
    plt.xticks([])
    plt.yticks([])
    plt.title(["ES", "MRS"][index], fontsize=10)

    plt.tight_layout()
plt.savefig("../pics/es_analysis.png", dpi=400)